In [25]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
import numpy as np

In [27]:
url = 'https://www.indeed.com/jobs?q=Data+Science&l=Los+Angeles%2C+CA&vjk=74cadd03cb0dd918'
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(url)

In [28]:
# all the scraping
job_titles = []
company_names = []
company_ratings = []
company_locations = []
job_pays = []
descriptions = []
posted = []
current_page = 1

pagination = driver.find_element(By.XPATH, ".//nav[contains(@role, 'navigation')]")
last_page = pagination.find_elements(By.XPATH, ".//a[contains(@data-testid, 'pagination-page-5')]")[-1].text
driver.set_page_load_timeout(10)

for page_num in range(1, int(last_page) + 18):
    container = driver.find_element(By.XPATH, ".//div[contains(@id, 'mosaic-jobResults')]")
    all_jobs = container.find_elements(By.XPATH, ".//div[contains(@class, 'job_seen_beacon')]")
    for job in all_jobs:
        title = job.find_element(By.XPATH, ".//h2[contains(@class, 'jobTitle')]").text
        try:
            location = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[1]
        except:
            location = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[0]
        company = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[0]
        when_posted= job.find_element(By.XPATH, ".//tr[contains(@class, 'underShelfFooter')]").text.split("\n")[-1].strip("More...")
        description = job.find_element(By.XPATH, ".//tr[contains(@class, 'underShelfFooter')]").text.split("\n")[0]
        try:
            string = job.find_element(By.XPATH, ".//div[contains(@class, 'salary-snippet-container')]").text
            listy = string.split()
            for element in listy:
                if "$" in element:
                    pay = element
        except:
            pay = "NaN"
        job_titles.append(title)
        job_pays.append(pay)
        company_names.append(company)
        descriptions.append(description)
        company_locations.append(location)
        posted.append(when_posted)
    page_num += 1

    try:
        #next = driver.find_element(By.XPATH, ".//a[contains(@aria-label, 'Next Page')]")
        next = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, ".//a[contains(@aria-label, 'Next Page')]")))
        print(next)
        next.click()
    except:
        #next = False
        print(f'failed {page_num}')
        break

<selenium.webdriver.remote.webelement.WebElement (session="d14c0d592f3cd923650755d0e61512cf", element="f.DD5B7A270F6E10B9B0A11F434CA325A3.d.C3A52FFF404B9C3BBE0EF166123F3128.e.1366")>
<selenium.webdriver.remote.webelement.WebElement (session="d14c0d592f3cd923650755d0e61512cf", element="f.DD5B7A270F6E10B9B0A11F434CA325A3.d.39E7D0212C88AE2777B85280065B25DE.e.5641")>
<selenium.webdriver.remote.webelement.WebElement (session="d14c0d592f3cd923650755d0e61512cf", element="f.DD5B7A270F6E10B9B0A11F434CA325A3.d.9C14D003ADC4E44FBE8182386D3EB200.e.10441")>
<selenium.webdriver.remote.webelement.WebElement (session="d14c0d592f3cd923650755d0e61512cf", element="f.DD5B7A270F6E10B9B0A11F434CA325A3.d.FC41031231868B8E081B9A22CA36B3EE.e.14890")>
<selenium.webdriver.remote.webelement.WebElement (session="d14c0d592f3cd923650755d0e61512cf", element="f.DD5B7A270F6E10B9B0A11F434CA325A3.d.C7033A3094CCD2D65260C0672E8959D7.e.18988")>
<selenium.webdriver.remote.webelement.WebElement (session="d14c0d592f3cd923650755d

In [31]:
# all the data cleaning
#separate into city and state

#try getting hybrid or remote status
def modality(location):
    if "Hybrid" in location or "hybrid" in location:
        return 'Hybrid'
    elif "Remote" in location or "remote" in location:
        return "Remote"
    elif "On-Site" in location or "on-site" in location:
        return 'On-Site'
    else:
        return np.nan
    

cities = []
states = []
modalities = []
# Parse location data
for location in company_locations:
    parts = location.split(', ')
    if len(parts) > 1:
        job_modality = modality(location)
        city = parts[0].replace("Hybrid", "")
        city = city.replace("remote", '')
        city = city.replace('in', '')
        state = parts[1] if len(parts) > 1 else None
        cities.append(city)
        states.append(state[:2])
        modalities.append(job_modality)
    else:
        cities.append(np.nan)
        states.append(np.nan)
        modalities.append(np.nan)


#make salaries integers
salaries = []
for salary in job_pays:
    if salary == "NaN" or salary == "" :
        salaries.append(np.nan)
    else:
        salary = salary.replace('$', '')
        salary = salary.replace(',', '')
        salary_list = salary.split('.')
        if len(salary_list[0]) <= 3:
            #Then I know it is hourly rate
            salaries.append(float(salary) * 40 * 52)
        else:
            salaries.append(float(salary))


job_info = pd.DataFrame({"Title": job_titles,
              "Company": company_names,
              "Location": company_locations,
              'Modality': modalities,
              "Job_Description": descriptions,
              "When_Posted": posted,
              "Pay": salaries,
              "City":cities,
               "State": states})


# fix locations and add ratings column
ratings = []
def fix_location(value):
    try:
        rating = float(value)
        ratings.append(rating)
        return np.nan
    except:
        ratings.append(np.nan)
        return value

job_info['Location'] = job_info["Location"].apply(fix_location)
job_info['Rating'] = ratings


In [32]:
job_info

,Title,Company,Location,Modality,Job_Description,When_Posted,Pay,City,State,Rating
0,Software Developer - AI Trainer (Contract),DataAnnotation,NaN,NaN,DataAnnotation is committed to creating qualit...,Active 12 days ag,83200.0,NaN,NaN,4.5
1,REMS Educator,Inizio Engage,NaN,NaN,Inizio Engage has a long-standing partnership ...,Posted 5 days ag,NaN,NaN,NaN,3.5
2,Data Engineer,LegalMation,"Hybrid remote in Los Angeles, CA 90071",Hybrid,About us LegalMation is a venture-backed AI Sa...,Active 4 days ag,80000.0,Los Angeles,CA,NaN
3,Associate Data Analyst - Hybrid Remote,CEDARS-SINAI,"Hybrid remote in Los Angeles, CA",Hybrid,Job Description Grow your career at Cedars-Sin...,Posted 4 days ago·,NaN,Los Angeles,CA,NaN
4,Data Analyst,UCLA,"Los Angeles, CA",NaN,About the California Policy Lab The California...,Posted 30+ days ago·,14583.0,Los Angeles,CA,NaN
...,...,...,...,...,...,...,...,...,...,...
309,Deals Tech & Data Solutions (Financial Analyti...,PRICE WATERHOUSE COOPERS,"Los Angeles, CA",NaN,And/or data visualization certifications such ...,Posted 30+ days ago·,NaN,Los Angeles,CA,NaN
310,Power Market Consultant - 1898 & Co. (Multiple...,Burns & McDonnell,"Los Angeles, CA",NaN,Conduct data analysis and interpretation of en...,Posted 17 days ago·,235000.0,Los Angeles,CA,NaN
311,"Senior Software Engineer, AI",Recruiting From Scratch,"Remote in Los Angeles, CA 90210",Remote,Building edge applications processing vision d...,Posted 30+ days ago·,220000.0,Remote Los Angeles,CA,NaN
312,Deals Tech & Data Solutions (Financial Analyti...,PRICE WATERHOUSE COOPERS,"Los Angeles, CA",NaN,And/or data visualization certifications such ...,Posted 30+ days ago·,NaN,Los Angeles,CA,NaN


In [36]:
#job_info.to_csv('3_26.csv', index=False)

In [37]:
job_info = pd.read_csv('3_26.csv')

In [38]:
job_info

,Unnamed: 0,Title,Company,Location,Modality,Job_Description,When_Posted,Pay,City,State,Rating
0,0,Software Developer - AI Trainer (Contract),DataAnnotation,NaN,NaN,DataAnnotation is committed to creating qualit...,Active 12 days ag,83200.0,NaN,NaN,4.5
1,1,REMS Educator,Inizio Engage,NaN,NaN,Inizio Engage has a long-standing partnership ...,Posted 5 days ag,NaN,NaN,NaN,3.5
2,2,Data Engineer,LegalMation,"Hybrid remote in Los Angeles, CA 90071",Hybrid,About us LegalMation is a venture-backed AI Sa...,Active 4 days ag,80000.0,Los Angeles,CA,NaN
3,3,Associate Data Analyst - Hybrid Remote,CEDARS-SINAI,"Hybrid remote in Los Angeles, CA",Hybrid,Job Description Grow your career at Cedars-Sin...,Posted 4 days ago·,NaN,Los Angeles,CA,NaN
4,4,Data Analyst,UCLA,"Los Angeles, CA",NaN,About the California Policy Lab The California...,Posted 30+ days ago·,14583.0,Los Angeles,CA,NaN
...,...,...,...,...,...,...,...,...,...,...,...
309,309,Deals Tech & Data Solutions (Financial Analyti...,PRICE WATERHOUSE COOPERS,"Los Angeles, CA",NaN,And/or data visualization certifications such ...,Posted 30+ days ago·,NaN,Los Angeles,CA,NaN
310,310,Power Market Consultant - 1898 & Co. (Multiple...,Burns & McDonnell,"Los Angeles, CA",NaN,Conduct data analysis and interpretation of en...,Posted 17 days ago·,235000.0,Los Angeles,CA,NaN
311,311,"Senior Software Engineer, AI",Recruiting From Scratch,"Remote in Los Angeles, CA 90210",Remote,Building edge applications processing vision d...,Posted 30+ days ago·,220000.0,Remote Los Angeles,CA,NaN
312,312,Deals Tech & Data Solutions (Financial Analyti...,PRICE WATERHOUSE COOPERS,"Los Angeles, CA",NaN,And/or data visualization certifications such ...,Posted 30+ days ago·,NaN,Los Angeles,CA,NaN


In [17]:
job_info

,Title,Company,Location,Job_Description,When_Posted,Pay,City,State,Modality,Rating


In [10]:
driver.quit()

In [34]:
#job_info.to_csv("masterpiece_fixed_pay.csv")